In [ ]:
import requests
import csv
import time

# URL для получения списка вакансий
URL_VAC = 'https://api.hh.ru/vacancies'

# ID Алтайского края
REGION_ID = 1217

# Цель - собрать данные о 1000 работодателях
MAX_EMPLOYERS = 1000

# Функция для получения данных о работодателях с открытыми вакансиями
def get_employers_with_vacancies(region_id, per_page=100):
    employers = {}  # Словарь для хранения данных о работодателях
    page = 0  # Номер текущей страницы

    # Цикл для сбора данных до достижения цели в 1000 работодателей
    while len(employers) < MAX_EMPLOYERS:
        # Запрос данных о вакансиях
        response = requests.get(URL_VAC, params={
            'area': region_id,
            'per_page': per_page,
            'page': page
        })

        # Проверка статуса ответа
        if response.status_code != 200:
            print(f"Error: {response.status_code}")
            break

        # Разбор ответа в формате JSON
        data = response.json()
        items = data.get('items', [])

        # Прерывание цикла, если вакансии закончились
        if not items:
            break

        # Обработка каждой вакансии
        for item in items:
            if 'employer' in item and 'id' in item['employer']:
                employer_id = item['employer']['id']
                employer_name = item['employer']['name']
                vacancy_url = item['alternate_url']

                # Добавление нового работодателя в словарь
                if employer_id not in employers:
                    employers[employer_id] = {
                        'id': employer_id,
                        'name': employer_name,
                        'vacancy_urls': [],
                        'vacancy_count': 0
                    }

                # Обновление данных о работодателе
                employers[employer_id]['vacancy_urls'].append(vacancy_url)
                employers[employer_id]['vacancy_count'] += 1

        # Переход к следующей странице
        page += 1
        # Небольшая пауза между запросами, чтобы избежать ограничения по частоте запросов
        time.sleep(0.2)

    return employers

# Функция для сохранения данных в CSV файл
def save_to_csv(employers, filename='employers.csv'):
    # Преобразование словаря в список и сортировка по количеству вакансий
    employer_list = list(employers.values())
    employer_list.sort(key=lambda x: x['vacancy_count'], reverse=True)

    # Запись данных в CSV файл
    with open(filename, 'w', encoding='utf-8', newline='') as f:
        fieldnames = ['id', 'name', 'vacancy_urls', 'vacancy_count']
        writer = csv.DictWriter(f, fieldnames=fieldnames)

        writer.writeheader()  # Запись заголовков
        for employer in employer_list:
            writer.writerow({
                'id': employer['id'],
                'name': employer['name'],
                'vacancy_urls': ', '.join(employer['vacancy_urls']),
                'vacancy_count': employer['vacancy_count']
            })

# Основная часть программы
if __name__ == '__main__':
    # Получение данных о работодателях
    employers = get_employers_with_vacancies(REGION_ID)

    # Проверка, удалось ли собрать данные о 1000 работодателях
    if len(employers) < MAX_EMPLOYERS:
        print(f"Found only {len(employers)} employers with open vacancies.")
    else:
        print(f"Collected data on {len(employers)} employers.")

    # Сохранение данных в CSV файл
    save_to_csv(employers)
    print(f'Data saved to employers.csv')